## Submission instructions

All code that you write should be in this notebook. Please include your names and student numbers. You have to submit this notebook, with your code and answers filled in. Make sure to add enough documentation.

For questions, make use of the "Lab" session (see schedule).
Questions can also be posted to the MS teams channel called "Lab".

**Note:** You are free to make use of Python libraries (e.g., numpy, sklearn, etc.) except any *fairness* libraries.

#### Name and student numbers
Jep Antonisse 3312070 Elias Hendriks 5930413 


## Dataset

In this assignment we are going to use the **COMPAS** dataset.

If you haven't done so already, take a look at this article: https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing.
For background on the dataset, see https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm.

**Reading in the COMPAS dataset**

The dataset can be downloaded here: https://github.com/propublica/compas-analysis/blob/master/compas-scores-two-years.csv

For this assignment, we focus on the protected attribute *race*.

The label (the variable we want to be able to predict) represents recidivism, which is defined as a new arrest within 2 years.

In [1]:
!wget -c https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd
compas_data = pd.read_csv('compas-scores-two-years.csv')

We apply several data preprocessing steps, including only retaining Caucasians and African Americans.

In [4]:
compas_data = compas_data[(compas_data.days_b_screening_arrest <= 30)
            & (compas_data.days_b_screening_arrest >= -30)
            & (compas_data.is_recid != -1)
            & (compas_data.c_charge_degree != 'O')
            & (compas_data.score_text != 'N/A')
            & ((compas_data.race == 'Caucasian') | (compas_data.race == 'African-American'))]

Take a look at the data:

In [5]:
print(compas_data.head())

    id              name      first    last compas_screening_date     sex  \
1    3       kevon dixon      kevon   dixon            2013-01-27    Male   
2    4          ed philo         ed   philo            2013-04-14    Male   
6    8     edward riddle     edward  riddle            2014-02-19    Male   
8   10  elizabeth thieme  elizabeth  thieme            2014-03-16  Female   
10  14    benjamin franc   benjamin   franc            2013-11-26    Male   

           dob  age       age_cat              race  ...  v_decile_score  \
1   1982-01-22   34       25 - 45  African-American  ...               1   
2   1991-05-14   24  Less than 25  African-American  ...               3   
6   1974-07-23   41       25 - 45         Caucasian  ...               2   
8   1976-06-03   39       25 - 45         Caucasian  ...               1   
10  1988-06-01   27       25 - 45         Caucasian  ...               4   

    v_score_text  v_screening_date  in_custody  out_custody  priors_count.1  \
1

Now take a look at the distribution of the protected attribute `race` and the distribution of our outcome variable `two_year_recid`.

**Note:** in the context of fair machine learning, the favorable label here is no recidivism, i.e., ```two_year_recid = 0```. So think about how what you will code as the positive class in your machine learning experiments, and make sure your interpretation of the results is consistent with this.

In [6]:
print('Number of instances per race category:')
print(compas_data[['race', 'two_year_recid']].value_counts())

Number of instances per race category:
race              two_year_recid
African-American  1                 1661
                  0                 1514
Caucasian         0                 1281
                  1                  822
Name: count, dtype: int64


## Data analysis

### **1. Exploration**

First we perform an exploratory analysis of the data.

**Question:** What is the size of the data? (i.e. how many data instances does it contain?)


In [7]:
# Your code
print('Number of data instances:')
print(compas_data.shape[0])

Number of data instances:
5278


**Question:** In the dataset, the protected attribute is `race`, which has two categories: White and African Americans. How many data instances belong to each category?

In [8]:
# Your code
print('Number of instances per race category:')
print(compas_data[['race']].value_counts())

Number of instances per race category:
race            
African-American    3175
Caucasian           2103
Name: count, dtype: int64


**Question:** What are the base rates (the probability of a favorable outcome for the two protected attribute classes)?

In [12]:
# Your code
def base_rate(attribute):
    grouped_by_a = compas_data.groupby(attribute)['two_year_recid']
    total_per_a = grouped_by_a.count()

    no_recid = grouped_by_a.apply(lambda x: (x == 0).sum())
    percentage_no_recid = (no_recid/total_per_a) * 100

    percentages = []
    print("Percentage of individuals with no recidivism (two_year_recid == 0):")
    for a in percentage_no_recid.index:
        print(f"{a}: {percentage_no_recid[a]:.2f}%")
        percentages.append(percentage_no_recid[a])
    return percentages

base_rate('race')
print("----")
base_rate('sex')
print()


Percentage of individuals with no recidivism (two_year_recid == 0):
African-American: 47.69%
Caucasian: 60.91%
----
Percentage of individuals with no recidivism (two_year_recid == 0):
Female: 63.82%
Male: 50.32%



**Question:** What are the base rates for the combination of both race and sex categories?

In [13]:
# Your code
base_rate(['race', 'sex'])
print()

Percentage of individuals with no recidivism (two_year_recid == 0):
('African-American', 'Female'): 63.02%
('African-American', 'Male'): 44.48%
('Caucasian', 'Female'): 64.73%
('Caucasian', 'Male'): 59.78%



**Question**

Write down a short interpretation of the statistics you calculated. What do you see?
> Answer:
- More black people are in jail
- White people have a lower percentage of recedivism.
- For both black and white people men have a lower percentage of recedivism.

### **2. Performance measures**

You will have to measure the performance and fairness of different classifiers in question 5. The performance will be calculated with the precision, recall, F1 and accuracy.
Additionally, you will have to calculate the statistical/demographic parity, the true positive rate (recall) and false positive rate per race group.

Make sure that you are able to calculate these metrics in the cell below.

In [16]:
# Your code for the performance measures
# Your code for the performance measures
def performance(predictions, true_labels):
    # predictions have different indices, so we set them anew
    predictions = predictions.reset_index(drop=True)
    true_labels = true_labels.reset_index(drop=True)
    
    # Since no recidivism is the positive label, we can compute the confusion matrix:
    tp = ((predictions['pred'] == 0) & (true_labels['two_year_recid'] == 0)).sum()
    fp = ((predictions['pred'] == 0) & (true_labels['two_year_recid'] == 1)).sum()

    tn = ((predictions['pred'] == 1) & (true_labels['two_year_recid'] == 1)).sum()
    fn = ((predictions['pred'] == 1) & (true_labels['two_year_recid'] == 0)).sum()

    # true positives / all positives
    precision = tp / (tp + fp)

    # true positives / all labels 0
    recall = tp / (tp + fn)

    # all true predictions / total intences
    accuracy = (tp + tn) / (tp + fp + tn + fn)

    f1 = 2 * (precision * recall) / (precision + recall)

    print(f"Accuracy: {100 * accuracy:.2f}%")
    print(f"Precision: {100 * precision:.2f}%")
    print(f"Recall: {100 * recall:.2f}%")
    print(f"F1-score: {f1:.2f}")
    return [accuracy, precision, recall, f1]

def group_performance(predictions, data, true_labels, attribute):

    data['pred'] = predictions['pred'].values #add the predictions to the datapoints
    data['two_year_recid'] = true_labels['two_year_recid'].values #add the true lables to the datapoints

    grouped_by_a = data.groupby(attribute)['pred'] #sort on 
    total_per_a = grouped_by_a.count()

    no_recid = grouped_by_a.apply(lambda x: (x == 0).sum())
    percentage_no_recid = (no_recid/total_per_a) * 100

    print("Percentage of individuals in a subgroup with no recidivism predicted:")
    percentages = []
    races = ['African-American', 'Caucasian']
    for a in percentage_no_recid.index:
        print(f"{races[a]}: {percentage_no_recid[a]:.2f}%")
        percentages.append(percentage_no_recid[a])
    print(f"statistical parity: {percentages[0]/percentages[1]:.2f}")

    black = data[data['race'] == 0]
    white = data[data['race'] == 1]

    print("\nDivided on sensitive attribute 'race':")
    print("\nAfrican-American:")
    performance(white[['pred']], white[['two_year_recid']])
    print("\nCaucasian:")
    performance(black[['pred']], black[['two_year_recid']])

    

### **3. Prepare the data**
For the classifiers in question 5, the input of the model can only contain numerical values, it is therefore important to convert the strings in the columns (features) of interest of the `compas_data` to floats or integers.

The columns of interest are features that you think will be informative or interesting in predicting the outcome variable.
Use the cell below to explore which of the Compas variables you need to convert to be able to use them for the classifiers.

Generate a new dataframe with your selected features in the right encoding (also make sure to include `two_year_recid`). You can implement this yourself, or use the `LabelEncoder` from `sklearn`.

**Note:** you do not need to convert all columns/features, only the ones you are interested in. However, do **not** include the feature `is_recid`.

In [18]:
# Your code to prepare the data
# Your code to prepare the data
from sklearn.preprocessing import LabelEncoder
print(compas_data.columns)


selected_features_categorical = ['age_cat', 'c_charge_degree', 'r_charge_degree', 'vr_charge_degree']
selected_features_numerical = ['decile_score', 'priors_count']


prepared_compas_data = pd.DataFrame()


# turn the strings into their exact integer value
for feature in selected_features_numerical:
    prepared_compas_data[feature] = compas_data[feature].astype(int)

# for categorical features, we have to assign them integer values
for feature in selected_features_categorical:
    le = LabelEncoder()
    prepared_compas_data[feature] = le.fit_transform(compas_data[feature])


# Protected Features
le = LabelEncoder()
prepared_compas_data['race'] = le.fit_transform(compas_data['race'])
le = LabelEncoder()
prepared_compas_data['sex'] = le.fit_transform(compas_data['sex'])


prepared_compas_data['two_year_recid'] = compas_data['two_year_recid'].astype(int)

le = LabelEncoder()
prepared_compas_data['race'] = le.fit_transform(compas_data['race'])
le = LabelEncoder()
prepared_compas_data['sex'] = le.fit_transform(compas_data['sex'])

prepared_compas_data.head()

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')


,decile_score,priors_count,age_cat,c_charge_degree,r_charge_degree,vr_charge_degree,race,sex,two_year_recid
1,3,0,0,0,3,2,0,1,1
2,4,4,2,0,6,8,0,1,1
6,6,14,0,0,2,8,1,1,1
8,1,0,0,1,9,8,1,0,0
10,4,0,0,0,9,8,1,1,0


**Question**

Give a short motivation (one-two sentence) per feature why you think this is informative or interesting to take into account.
> Answer:

### **4. Train and test split**

Divide the dataset into a train (80%) and test split (20%), either by implementing it yourself, or by using an existing library.

**Note:** Usually when carrying out machine learning experiments,
we also need a dev set for developing and selecting our models (incl. tuning of hyper-parameters).
However, in this assignment, the goal is not to optimize
the performance of models so we'll only use a train and test split.




In [ ]:
# Your code to split the data


### **5. Classifiers**

Now, train and test different classifiers and report the following statistics:

* Overall performance:

  * Precision
  * Recall
  * F1
  * Accuracy

* Fairness performance:

  * The statistical parity difference for the protected attribute `race`(i.e. the difference in the probability of receiving a favorable label between the two protected attribute groups);
  * The true positive rates of the two protected attribute groups
  * The false positive rates of the two protected attribute groups.

For training the classifier we recommend using scikit-learn (https://scikit-learn.org/stable/).

#### **5.1 Regular classification**
Train a logistic regression classifier with the race feature and all other features that you are interested in.

In [ ]:
# Your code for classifier 1


#### **5.2 Without the protected attribute**
Train a logistic regression classifier without the race feature, but with all other features you used in 5.1.


In [ ]:
# Your code for classifier 2


**Question**

Write down a short interpretation of the results you calculated. What do you see?
> Answer:

#### **5.3 Pre-processing: Reweighing**
Train and test a classifier with weights (see lecture slide for the weight calculation)

In [ ]:
# Your code for classifier 3


**Question**

 Report the 4 weights that are used for reweighing and a short **interpretation/discussion** of the weights and the classifier results.
> Answer:


#### **5.4 Post-processing: Equalized odds**
Use the predictions by the first classifier for this post processing part (see lecture slides for more information about post processing for equalized odds).

We have the following parameters (A indicates group membership, Y_{hat} the original prediction, Y_{tilde} the prediction of the derived predictor).

* `p_00` = P(Y_{tilde} = 1 | Y_{hat} = 0 & A = 0)
* `p_01` = P(Y_{tilde} = 1 | Y_{hat} = 0 & A = 1)
* `p_10` = P(Y_{tilde} = 1 | Y_{hat} = 1 & A = 0)
* `p_11` = P(Y_{tilde} = 1 | Y_{hat} = 1 & A = 1)


Normally, the best parameters `p_00, p_01, p_10, p_11` are found with a linear program that minimizes loss between predictions of a derived predictor and the actual labels. In this assignment we will not ask you to do this. Instead, we would like you to follow the next steps to find parameters, post-process the data and check the performance of this classifier with post-processing:

1. Generate 5000 different samples of these 4 parameters randomly;
2. Write a function (or more) that applies these 4 parameters to postprocess the predictions.
3. For each generated set of 4 parameters:
  - Change the predicted labels with the function(s) from step 2;
  - Evaluate these 'new' predictions, by calculating group-wise TPR and FPR, as well as overall performance based on F1 and/or accuracy.
4. Choose the best set of parameters. Take into account the equalized odds fairness measure, as well a performance measure like accuracy or F1.
5. Check the overall performance (precision, recall, accuracy, F1, etc.) of the new predictions after post-processing.

In [ ]:
# Your code for step 1
import random

random_parameters = []
for _ in range(1000):
  p_00 = ...
  p_01 = ...
  p_10 = ...
  p_11 = ...
  random_parameters.append({(0, 0): p_00,
                            (0, 1): p_01,
                            (1, 0): p_10,
                            (1, 1): p_11})

# Example, first set of random parameters
print(random_parameters[0])

In [ ]:
# Your code for step 2
# Create a dataframe with the necessary information
df_post_data = pd.DataFrame({'race_num': ...,
                             'pred_labels': ...,
                             'true_labels': ...})

# the number of cases falling in each condition
subset_sizes = {
    (0, 0): len(df_post_data.query('pred_labels == 0 & race_num == 0')),
    (0, 1): len(df_post_data.query('pred_labels == 0 & race_num == 1')),
    (1, 0): len(df_post_data.query('pred_labels == 1 & race_num == 0')),
    (1, 1): len(df_post_data.query('pred_labels == 1 & race_num == 1'))

}

def generate_labels(subset_sizes, p_dict):
    """
    subset_sizes: dict with number of cases falling in each condition
    p_dict: the postprocessing parameters
    """
    new_predictions = {}

    for (prediction, group), p in p_dict.items():

      # The number of instances for which we need to generate labels
      num_instances = subset_sizes[(prediction, group)]

      # Write your code here.

      # save the new predictions
      new_predictions[(prediction, group)] = ...

    return new_predictions

In [ ]:
# Your code for step 3

for p_dict in random_parameters:

  new_predictions = generate_labels(subset_sizes, p_dict)

  # replace the predictions
  df_copy = df_post_data.copy()

  for (pred, group), p in p_dict.items():

    new_preds = new_predictions[(pred,group)]
    df_copy.loc[(df_post_data['pred_labels'] == pred) &
                (df_post_data['race_num'] == group), 'pred_labels'] = new_preds


  # evaluate the new predictions and save the scores
  # Write your code here.


In [ ]:
# Your code for step 4 and 5


**Question**

Describe how you selected the best set of parameters. Furthermore, how do you interpret the best set of parameters that you found? And what do you think of the results of the new classifier?
>Answer

#### **Overall discussion**
For all 4 classifiers that you trained, describe:
- Does this classifier satisfies statistical parity?
- Does the classifier satisfy the equal opportunity criterion?

Finally, how do the different classifiers compare against each other?

>Answer

### **6. Intersectional fairness**
In the questions above `race` was the only protected attribute. However, multiple protected attributes sometimes interact, leading to different fairness outcomes for different combinations of these protected attributes.

Now explore the intersectional fairness for protected attributes `race` and `sex` for the first two classifiers from question 5. Make a combination of the `race` and `sex` column, resulting in four new subgroups (e.g., female Caucasian), and report the maximum difference between the subgroups for statistical parity, TPR and FPR.
For example, suppose we have four groups with TPRs 0.1, 0.2, 0.3, 0.8, then the maximum difference is 0.7.

Your code to evaluate intersectional fairness for Classifier 1:




In [ ]:
# Your code for intersectional fairness

Your code to evaluate intersectional fairness for Classifier 2:


In [ ]:
# Your code for intersectional fairness

**Question**

Write down a short interpretation of the results you calculated. What do you see?
> Answer:

## Discussion
Provide a short ethical discussion (1 or 2 paragraphs) reflecting on these two aspects:

1) The use of a ML system to try to predict recidivism;

2) The public release of a dataset like this.

> Answer